# Week 4 - Models and Experimentation

## Step 1 Training a model

For the purposes of this demo, we will be using this [adapted demo](https://www.datacamp.com/tutorial/xgboost-in-python) and training an XGBoost model, and then doing some experimentation and hyperparameter tuning.


If running this notebook locally, use the following steps to create virtual environment:
- Don't use past python 3.10
- To create virtual environment use "venv"

`python -m venv NAME`

- Try to avoid anaconda, poetry or similar package management platforms
- To install a package use pip

`python -m pip install <package-name>`

- once you are done working with this virtual environment, deactivate it with `deactivate`

### Install packages

# Experimentation Overview  

For this experiment, we are trying to tune the hyper parameters for the XG Boost algorithm. We are using the WandB logging to track the experiments.

Hyper-parameters chosen to tune :
1. Learning Rate : [0.01,0.03, 0.1, 0.3, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.2, 1.3, 1.4, 1.5 ]
2. Max Depth : [1,2,3,4,5,6]


- W and B Result link : https://api.wandb.ai/links/zero-monster/1s6p85rk
- Best Parameters : LR = 0.01 , Max Depth = 6

- Best Performance : RMSE on Validation Set = 533.2
- Original Best Performance : 549.10
- Improvement in Performance : 2%

In [2]:
!pip install wandb -qU

In [2]:
import xgboost as xgb
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


### Import data

We will be using Diamonds dataset imported from Seaborn. It is also available on [Kaggle](https://www.kaggle.com/datasets/shivam2503/diamonds).

Read about the features by following the link. We will be predicting the price of diamonds.

In [3]:
diamonds = sns.load_dataset('diamonds')
diamonds.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [5]:
diamonds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   carat    53940 non-null  float64 
 1   cut      53940 non-null  category
 2   color    53940 non-null  category
 3   clarity  53940 non-null  category
 4   depth    53940 non-null  float64 
 5   table    53940 non-null  float64 
 6   price    53940 non-null  int64   
 7   x        53940 non-null  float64 
 8   y        53940 non-null  float64 
 9   z        53940 non-null  float64 
dtypes: category(3), float64(6), int64(1)
memory usage: 3.0 MB


In [6]:
diamonds.shape

(53940, 10)

In [4]:
X,y = diamonds.drop('price', axis=1), diamonds[['price']]

# For the cut, color and clarity use pandas category to enable XGBoost ability to deal with categorical data.

X['cut'] = X['cut'].astype('category')
X['color'] = X['color'].astype('category')
X['clarity'] = X['clarity'].astype('category')

### Split the data and train a model

In [5]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create DMatrix
dtrain = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)
dtest = xgb.DMatrix(X_test, label=y_test, enable_categorical=True)

In [6]:
# Define hyperparameters
params = {"objective": "reg:squarederror", "tree_method": "gpu_hist"}

n = 100
model = xgb.train(
   params=params,
   dtrain=dtrain,
   num_boost_round=n,
)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [16:07:06] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


In [7]:
# Define evaluation metrics - Root Mean Squared Error

predictions = model.predict(dtest)
rmse = mean_squared_error(y_test, predictions, squared=False)
print(f"RMSE: {rmse}")

RMSE: 532.8838153117543


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [16:07:10] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


### Incorporate validation

In [11]:
params = {"objective": "reg:squarederror", "tree_method": "gpu_hist"}
n = 100

# Create the validation set
evals = [(dtrain, "train"), (dtest, "validation")]

In [12]:
evals = [(dtrain, "train"), (dtest, "validation")]

model = xgb.train(
   params=params,
   dtrain=dtrain,
   num_boost_round=n,
   evals=evals,
   verbose_eval=10,
)

[0]	train-rmse:2859.49097	validation-rmse:2851.62630
[10]	train-rmse:550.99470	validation-rmse:571.16640
[20]	train-rmse:491.51435	validation-rmse:544.08058


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [15:31:06] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


[30]	train-rmse:464.38845	validation-rmse:537.01895
[40]	train-rmse:445.99106	validation-rmse:533.85127
[50]	train-rmse:430.36010	validation-rmse:532.90320
[60]	train-rmse:418.87898	validation-rmse:533.04629
[70]	train-rmse:409.66247	validation-rmse:533.58046
[80]	train-rmse:397.34048	validation-rmse:534.31963
[90]	train-rmse:389.94294	validation-rmse:532.61946
[99]	train-rmse:377.70831	validation-rmse:532.88383


In [13]:
# Incorporate early stopping
n = 10000


model = xgb.train(
   params=params,
   dtrain=dtrain,
   num_boost_round=n,
   evals=evals,
   verbose_eval=50,
   # Activate early stopping
   early_stopping_rounds=50
)

[0]	train-rmse:2859.49097	validation-rmse:2851.62630
[50]	train-rmse:430.36010	validation-rmse:532.90320
[100]	train-rmse:377.56825	validation-rmse:532.79980
[103]	train-rmse:375.44970	validation-rmse:532.50220


In [14]:
# Cross-validation

params = {"objective": "reg:squarederror", "tree_method": "gpu_hist"}
n = 1000

results = xgb.cv(
   params, dtrain,
   num_boost_round=n,
   nfold=5,
   early_stopping_rounds=20
)


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [15:31:07] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


In [15]:
results.head()

,train-rmse-mean,train-rmse-std,test-rmse-mean,test-rmse-std
0,2861.153015,8.266765,2861.773555,36.937516
1,2081.378004,5.534608,2084.973481,32.064109
2,1545.361682,3.287745,1553.681211,31.059209
3,1182.364236,3.585787,1192.464771,26.157805
4,941.828819,2.971779,958.467497,23.613538


In [16]:
best_rmse = results['test-rmse-mean'].min()

best_rmse

549.1039652582465

## Start W&B


- Login into your W&B profile using the code below
- Alternatively you can set environment variables. There are several env variables which you can set to change the behavior of W&B logging. The most important are:
    - WANDB_API_KEY - find this in your "Settings" section under your profile
    - WANDB_BASE_URL - this is the url of the W&B server

- Find your API Token in "Profile" -> "Setttings" in the W&B App



In [17]:
# Log in to your W&B account
import wandb

wandb.login()

wandb: Currently logged in as: bhavin-rathava-bwin (zero-monster). Use `wandb login --relogin` to force relogin


True

In [18]:
# TO DO
# Start experiment tracking with W&B
# Do at least 5 experiments with various hyperparameters
# Choose any method for hyperparameter tuning: grid search, random search, bayesian search
# Describe your findings and what you see

In [19]:
from sklearn.model_selection import GridSearchCV

wandb.init(
    # set the wandb project where this run will be logged
    project="Practicum Experimentation Week 4",
    config = {
        "Logging": "RMSE ERROR",
        "MODEL TYPE": "XGBOOST",
        "Hyper Parameter": "Learning Rate"
    }
)

learning_rate = np.random.rand(10)


for learning_ in [0.01,0.03, 0.1, 0.3, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.2, 1.3, 1.4, 1.5 ]:
    params = {"objective": "reg:squarederror", "tree_method": "gpu_hist", "learning_rate" : learning_}
    model = xgb.train(
    params=params,
    dtrain=dtrain,
    num_boost_round=n,
    evals=evals
    )

    predictions = model.predict(dtest)
    rmse = mean_squared_error(y_test, predictions, squared=False)

    wandb.log({"Learning Rate": learning_, "RMSE_Loss": rmse})

[0]	train-rmse:3951.92462	validation-rmse:3949.02748
[1]	train-rmse:3914.29025	validation-rmse:3911.21998
[2]	train-rmse:3877.10275	validation-rmse:3873.86672
[3]	train-rmse:3840.30732	validation-rmse:3836.90210
[4]	train-rmse:3803.89438	validation-rmse:3800.32256
[5]	train-rmse:3767.86467	validation-rmse:3764.12914
[6]	train-rmse:3732.20828	validation-rmse:3728.30718
[7]	train-rmse:3696.92823	validation-rmse:3692.85775
[8]	train-rmse:3662.02397	validation-rmse:3657.78952
[9]	train-rmse:3627.46912	validation-rmse:3623.08587
[10]	train-rmse:3593.28635	validation-rmse:3588.73717
[11]	train-rmse:3559.44161	validation-rmse:3554.71475
[12]	train-rmse:3525.97874	validation-rmse:3521.06781
[13]	train-rmse:3492.86570	validation-rmse:3487.77529
[14]	train-rmse:3460.11145	validation-rmse:3454.83778


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [15:31:12] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


[15]	train-rmse:3427.65383	validation-rmse:3422.21992
[16]	train-rmse:3395.56677	validation-rmse:3389.96690
[17]	train-rmse:3363.75667	validation-rmse:3358.08757
[18]	train-rmse:3332.31965	validation-rmse:3326.47775
[19]	train-rmse:3301.17180	validation-rmse:3295.26749
[20]	train-rmse:3270.41375	validation-rmse:3264.35438
[21]	train-rmse:3239.92519	validation-rmse:3233.88165
[22]	train-rmse:3209.73054	validation-rmse:3203.59201
[23]	train-rmse:3179.80215	validation-rmse:3173.60675
[24]	train-rmse:3150.18892	validation-rmse:3144.01912
[25]	train-rmse:3120.92669	validation-rmse:3114.64985
[26]	train-rmse:3091.93693	validation-rmse:3085.54884
[27]	train-rmse:3063.14791	validation-rmse:3056.74740
[28]	train-rmse:3034.75572	validation-rmse:3028.31985
[29]	train-rmse:3006.57676	validation-rmse:3000.08035
[30]	train-rmse:2978.78205	validation-rmse:2972.18011
[31]	train-rmse:2951.11499	validation-rmse:2944.49426
[32]	train-rmse:2923.86171	validation-rmse:2917.20940
[33]	train-rmse:2896.92357	v

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [15:31:21] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [15:31:21] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


[17]	train-rmse:2399.75751	validation-rmse:2392.73783
[18]	train-rmse:2334.92635	validation-rmse:2328.14500
[19]	train-rmse:2272.34291	validation-rmse:2265.94534
[20]	train-rmse:2211.58871	validation-rmse:2205.37752
[21]	train-rmse:2152.81578	validation-rmse:2146.73568
[22]	train-rmse:2095.88970	validation-rmse:2089.98593
[23]	train-rmse:2040.79954	validation-rmse:2034.90674
[24]	train-rmse:1987.38858	validation-rmse:1981.52309
[25]	train-rmse:1935.58414	validation-rmse:1929.56333
[26]	train-rmse:1885.33063	validation-rmse:1879.42155
[27]	train-rmse:1836.75235	validation-rmse:1830.87821
[28]	train-rmse:1790.06353	validation-rmse:1784.05102
[29]	train-rmse:1744.38750	validation-rmse:1738.34204
[30]	train-rmse:1700.40420	validation-rmse:1694.41990
[31]	train-rmse:1657.88109	validation-rmse:1652.06219
[32]	train-rmse:1616.42909	validation-rmse:1610.79635
[33]	train-rmse:1576.28755	validation-rmse:1570.73065
[34]	train-rmse:1537.64826	validation-rmse:1532.03750
[35]	train-rmse:1500.24850	v

...

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [15:33:16] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [15:33:16] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


[13]	train-rmse:553.81830	validation-rmse:709.28330
[14]	train-rmse:550.02216	validation-rmse:712.79736
[15]	train-rmse:542.89987	validation-rmse:717.05506
[16]	train-rmse:535.44238	validation-rmse:721.61541
[17]	train-rmse:530.02217	validation-rmse:723.70083
[18]	train-rmse:525.65447	validation-rmse:729.32842
[19]	train-rmse:524.01731	validation-rmse:728.59179
[20]	train-rmse:518.09794	validation-rmse:729.72151
[21]	train-rmse:511.75337	validation-rmse:731.01028
[22]	train-rmse:507.14502	validation-rmse:737.69910
[23]	train-rmse:501.85109	validation-rmse:737.84057
[24]	train-rmse:494.16009	validation-rmse:740.88080
[25]	train-rmse:488.95273	validation-rmse:743.10089
[26]	train-rmse:483.69956	validation-rmse:739.80747
[27]	train-rmse:479.09845	validation-rmse:739.09008
[28]	train-rmse:475.90155	validation-rmse:743.66176
[29]	train-rmse:470.37327	validation-rmse:742.65028
[30]	train-rmse:466.76824	validation-rmse:745.06672
[31]	train-rmse:463.43831	validation-rmse:745.52624
[32]	train-r

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [15:33:27] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [15:33:27] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


[15]	train-rmse:549.03911	validation-rmse:722.22679
[16]	train-rmse:540.86606	validation-rmse:722.01172
[17]	train-rmse:534.17797	validation-rmse:722.25443
[18]	train-rmse:528.03425	validation-rmse:722.81126
[19]	train-rmse:522.12570	validation-rmse:733.14451
[20]	train-rmse:517.62842	validation-rmse:734.34435
[21]	train-rmse:512.99695	validation-rmse:738.49378
[22]	train-rmse:507.14087	validation-rmse:738.29062
[23]	train-rmse:501.95816	validation-rmse:737.26503
[24]	train-rmse:497.07403	validation-rmse:738.70110
[25]	train-rmse:492.55389	validation-rmse:740.06291
[26]	train-rmse:489.04221	validation-rmse:740.90728
[27]	train-rmse:486.16750	validation-rmse:744.17010
[28]	train-rmse:481.91837	validation-rmse:742.24585
[29]	train-rmse:477.68831	validation-rmse:742.83934
[30]	train-rmse:472.44020	validation-rmse:743.84049
[31]	train-rmse:471.27908	validation-rmse:744.24822
[32]	train-rmse:467.88311	validation-rmse:747.56274
[33]	train-rmse:465.82314	validation-rmse:745.97170
[34]	train-r

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [15:33:38] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


In [8]:
# As evident from the Learning Rate Vs RMSE Loss, we can see that on higher values of LR, the RMSE is increasing monotonically meaning that the model is overfitting

bestLR = 0.01
bestDepth = 6

params = {"objective": "reg:squarederror", "tree_method": "gpu_hist", "learning_rate" : bestLR, "max_depth" : bestDepth}

n = 1000

results = xgb.cv(
   params, dtrain,
   num_boost_round=n,
   nfold=5,
   early_stopping_rounds=20
)

print(results.head())

print(f"Best RMSE : {results['test-rmse-mean'].min()}")
# https://wandb.ai/zero-monster/Practicum%20Experimentation%20Week%204/reports/RMSE_Loss-24-04-17-18-43-53---Vmlldzo3NTkwMzkw

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [16:07:29] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


   train-rmse-mean  train-rmse-std  test-rmse-mean  test-rmse-std
0      3951.951893       10.524378     3951.843391      42.362551
1      3914.374514       10.448655     3914.268714      42.156624
2      3877.186551       10.370155     3877.079497      41.947674
3      3840.389114       10.296209     3840.289917      41.740940
4      3803.986712       10.234358     3803.909143      41.537761
Best RMSE : 533.2951820146366
